In [1]:
import warnings
warnings.filterwarnings("ignore", category=RuntimeWarning)

import sys
import os
project_root = os.path.abspath(os.path.join(os.getcwd(), os.pardir))
if project_root not in sys.path: sys.path.insert(0, project_root)


In [2]:
import os
import numpy as np
import pandas as pd

import src.data_handling as data_handling

# paths
PRODUCTION_MODEL_FOLDER_PATH = 'models/production'
DFN_FILE_PATH = os.path.join(PRODUCTION_MODEL_FOLDER_PATH, 'dfn_best.pth')
GBM_FILE_PATH =  os.path.join(PRODUCTION_MODEL_FOLDER_PATH, 'gbm_best.pth')
EN_FILE_PATH = os.path.join(PRODUCTION_MODEL_FOLDER_PATH, 'en_best.pth')

PREPROCESSOR_PATH = 'preprocessors/column_transformer.pkl'

file_name = 'online_retail.csv'
file_path = os.path.join(project_root, 'data', 'raw', file_name)
df = pd.read_csv(file_path)
df = data_handling.scripts.sanitize_column_names(df=df)
df = data_handling.scripts.structure_missing_values(df=df)

In [3]:
negative_sales_items = df[df['quantity'] < 0]
print(negative_sales_items)

       invoiceno stockcode                       description  quantity  \
141      C536379         D                          Discount        -1   
154      C536383    35004C   SET OF 3 COLOURED  FLYING DUCKS        -1   
235      C536391     22556    PLASTERS IN TIN CIRCUS PARADE        -12   
236      C536391     21984  PACK OF 12 PINK PAISLEY TISSUES        -24   
237      C536391     21983  PACK OF 12 BLUE PAISLEY TISSUES        -24   
...          ...       ...                               ...       ...   
540449   C581490     23144   ZINC T-LIGHT HOLDER STARS SMALL       -11   
541541   C581499         M                            Manual        -1   
541715   C581568     21258        VICTORIAN SEWING BOX LARGE        -5   
541716   C581569     84978  HANGING HEART JAR T-LIGHT HOLDER        -1   
541717   C581569     20979     36 PENCILS TUBE RED RETROSPOT        -5   

            invoicedate  unitprice  customerid         country  
141      12/1/2010 9:41      27.50     14527.0

In [4]:
print(df[df['invoiceno'] == 'C536379'])

    invoiceno stockcode description  quantity     invoicedate  unitprice  \
141   C536379         D    Discount        -1  12/1/2010 9:41       27.5   

     customerid         country  
141     14527.0  United Kingdom  


In [5]:
# irregular stockcodes
short_stockcodes = df[df['stockcode'].str.len() < 5]['stockcode'].unique()
print(short_stockcodes)

# D = Discount
# M = Manual (sales)
# m = M (typo)
# S = Samples
# B = Bad debt
# C2 = Cariage (transport)
# DOT = Dotcom postage
# POST = Postage
# PADS = PADS TO MATCH ALL CUSHIONS
# CRUK = CRUK Commission (commission fee paid)

['POST' 'D' 'C2' 'DOT' 'M' 'S' 'm' 'PADS' 'B' 'CRUK']


In [6]:
# # check what s inside
# print(df[df['stockcode'] == 'S'])

In [7]:
# replace typo m to M
df['stockcode'] = df['stockcode'].replace('m', 'M')

short_stockcodes = df[df['stockcode'].str.len() < 5]['stockcode'].unique()
print(short_stockcodes)

['POST' 'D' 'C2' 'DOT' 'M' 'S' 'PADS' 'B' 'CRUK']


In [8]:
# drop irrelevant stockcodes to sales (B, CRUK, C2)
stockcodes_to_drop = ['B', 'CRUK', 'C2']
df = df[~df['stockcode'].isin(stockcodes_to_drop)]

short_stockcodes = df[df['stockcode'].str.len() < 5]['stockcode'].unique()
print(short_stockcodes)

['POST' 'D' 'DOT' 'M' 'S' 'PADS']


In [9]:
df.head()

,invoiceno,stockcode,description,quantity,invoicedate,unitprice,customerid,country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850.0,United Kingdom


In [10]:
# remove unnecessary features
if 'description' in df.columns.tolist(): df = df.drop(columns='description')

# adds quantity momentum features
df['invoicedate'] = pd.to_datetime(df['invoicedate'], errors='coerce')
df['year'] = df['invoicedate'].dt.year
df['year_month'] = df['invoicedate'].dt.to_period('M')
df['month_name'] = df['invoicedate'].dt.strftime('%b')
df['day_of_week'] = df['invoicedate'].dt.strftime('%a')
df['sales'] = df['quantity'] * df['unitprice']

_df_prod_month_agg = df.copy().groupby(['stockcode', 'year_month']).agg(
    prod_total_monthly_sales=('sales', 'sum'),
    prod_ave_monthly_price=('unitprice', 'mean')
).reset_index().sort_values(by=['stockcode', 'year_month'])
_df_prod_month_agg['product_avg_sales_last_month'] = _df_prod_month_agg.groupby('stockcode')['prod_total_monthly_sales'].shift(1)
_df_prod_last_month_agg = _df_prod_month_agg.groupby('stockcode')['product_avg_sales_last_month'].mean().reset_index()
_df_prod_last_month_agg_renamed = _df_prod_last_month_agg.rename(
    columns={'product_avg_sales_last_month': 'new_product_avg_sales_last_month'}
)

df_fin = pd.merge(
    df,
    _df_prod_last_month_agg_renamed[['stockcode', 'new_product_avg_sales_last_month']],
    on='stockcode',
    how='left'
)
df_fin['product_avg_sales_last_month'] = df_fin['new_product_avg_sales_last_month']
df_fin = df_fin.drop(columns='new_product_avg_sales_last_month', axis=1)
df_fin['product_avg_sales_last_month'] = df_fin['product_avg_sales_last_month'].fillna(value=0)


# add customer related features
# handle customer registration
df_fin['is_registered'] = np.where(df_fin['customerid'].isna(), 0, 1)
df_fin['customerid'] = df_fin['customerid'].fillna('unknown').astype('str')

## 1. customer_recency_days
_df_all_customers_year_month = pd.MultiIndex.from_product(
    [df_fin['customerid'].unique(), df_fin['year_month'].unique()], # type: ignore
    names=['customerid', 'year_month']
).to_frame(index=False).sort_values(by=['customerid', 'year_month']).reset_index(drop=True)
_df_customer_monthly_agg = df_fin.copy().groupby(['customerid', 'year_month']).agg(
    monthly_sales=('sales', 'sum'),
    monthly_unique_invoices=('invoiceno', 'nunique'),
    monthly_last_purchase_date=('invoicedate', 'max')
).reset_index()
_df_cus = _df_all_customers_year_month.merge(
    _df_customer_monthly_agg,
    on=['customerid', 'year_month'],
    how='left'
).sort_values(by=['customerid', 'year_month'])

_df_cus['pfin_last_purchase_date'] = _df_cus.groupby('customerid')['monthly_last_purchase_date'].shift(1)
_df_cus['invoice_timestamp_end'] = _df_cus['year_month'].dt.end_time
_df_cus['customer_recency_days'] = (_df_cus['invoice_timestamp_end'] - _df_cus['pfin_last_purchase_date']).dt.days
df_fin['customer_recency_days'] = _df_cus['customer_recency_days']
max_recency = _df_cus['customer_recency_days'].max()
df_fin['customer_recency_days'] = df_fin['customer_recency_days'].fillna(value=max_recency + 30)
df_fin['customer_recency_days'] = df_fin['customer_recency_days'].fillna(365)

## 2. customer_total_spend_ltm
if not _df_cus['customerid'].isna().all():
    _df_cus['customer_total_spend_ltm'] = _df_cus.groupby('customerid')['monthly_sales'].rolling(window=3, closed='left').sum().reset_index(level=0, drop=True)
    df_fin['customer_total_spend_ltm'] = _df_cus['customer_total_spend_ltm']
    df_fin['customer_total_spend_ltm'] = df_fin['customer_total_spend_ltm'].fillna(value=0)

    ## 3. customer_freq_ltm
    _df_cus['customer_freq_ltm'] = _df_cus.groupby('customerid')['monthly_unique_invoices'].rolling(window=3, closed='left').sum().reset_index(level=0, drop=True)
    df_fin['customer_freq_ltm'] = _df_cus['customer_freq_ltm']
    df_fin['customer_freq_ltm'] = df_fin['customer_freq_ltm'].fillna(value=0)
else:
    df_fin['customer_freq_ltm'] = 0
    df_fin['customer_total_spend_ltm'] = 0




# after computing customer sales features, drop unnecessary data
stockcodes_to_drop = ['D', 'S']
df_fin = df_fin[~df_fin['stockcode'].isin(stockcodes_to_drop)]
df_fin = df_fin.drop(columns=['month_name', 'quantity'], axis='columns')

# sales (drop negative vals)
df_fin['sales'] = pd.to_numeric(df_fin['sales'], errors='coerce')
df_fin['sales'] = df_fin['sales'].fillna(0)
df_fin = df_fin[df_fin['sales'] > 0]


# dtype transformation
df_fin['year_month'] = df_fin['year_month'].dt.month
df_fin['invoicedate'] = df_fin['invoicedate'].astype(int) / 10 ** 9


# imputation
df_fin['customerid'] = df_fin['customerid'].fillna(value='unknown')
df_fin['stockcode'] = df_fin['stockcode'].fillna(value='unknown')
df_fin['invoiceno'] = df_fin['invoiceno'].fillna(value='unknown')
# df_fin['quantity'] = df_fin['quantity'].fillna(value=0)

# imputation (values referred to stockcode)
df_imputed = df_fin.copy().sort_values(by='stockcode').reset_index(drop=True)
df_stockcode = df_imputed.groupby('stockcode', as_index=False).agg(
    imputed_country=('country', lambda x: x.mode().iloc[0] if not x.mode().empty else 'unknown'),
    imputed_unitprice=('unitprice', 'median')
)
df_fin = pd.merge(df_fin, df_stockcode, on='stockcode', how='left')
df_fin['country'] = df_fin['country'].fillna(df_fin['imputed_country'])

global_median = df_fin['unitprice'].median()
df_fin['unitprice'] = df_fin['unitprice'].fillna(df_fin['imputed_unitprice'])
df_fin['unitprice'] = df_fin['unitprice'].fillna(global_median)
df_fin = df_fin.drop(columns=['imputed_country', 'imputed_unitprice'])






# df_fin['is_return'] = (df_fin['sales'] < 0).astype(int)

# drop negative sales (return etc)
# df_fin_log = df_fin.copy()
# df_fin_log = df_fin_log[df_fin_log['sales'] > 0]
# df_fin_log['sales'] = np.where(df_fin_log['is_return'] == 1, 0, df_fin_log['sales'])

# transform sales to logged values
alpha = 1e-3
df_fin['sales'] = np.log(df_fin['sales'] + alpha)



In [11]:
df = df_fin.copy()

In [12]:
from sklearn.model_selection import train_test_split

# classify num and cat columns
target_col = 'sales'

num_cols, cat_cols = data_handling.scripts.categorize_num_cat_cols(df=df, target_col=target_col)
if cat_cols: 
    for col in cat_cols: df[col] = df[col].astype('string')


# creates train, val, test datasets
y = df[target_col]
X = df.copy().drop(target_col, axis='columns')

test_size, random_state = 50000, 42
X_tv, X_test, y_tv, y_test = train_test_split(X, y, test_size=test_size, random_state=random_state, shuffle=True)
X_train, X_val, y_train, y_val = train_test_split(X_tv, y_tv, test_size=test_size, random_state=random_state, shuffle=True)
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 429960 entries, 54212 to 320071
Data columns (total 14 columns):
 #   Column                        Non-Null Count   Dtype  
---  ------                        --------------   -----  
 0   invoiceno                     429960 non-null  string 
 1   stockcode                     429960 non-null  string 
 2   invoicedate                   429960 non-null  float64
 3   unitprice                     429960 non-null  float64
 4   customerid                    429960 non-null  string 
 5   country                       429960 non-null  string 
 6   year                          429960 non-null  string 
 7   year_month                    429960 non-null  string 
 8   day_of_week                   429960 non-null  string 
 9   product_avg_sales_last_month  429960 non-null  float64
 10  is_registered                 429960 non-null  string 
 11  customer_recency_days         429960 non-null  float64
 12  customer_total_spend_ltm      429960 non-null

In [13]:
X_train, X_val, X_test, preprocessor = data_handling.scripts.transform_input(X_train, X_val, X_test, num_cols=num_cols, cat_cols=cat_cols)

import src.model.torch_model as t

file_path = os.path.join(project_root, 'models', 'production', 'dfn_best.pth')
model = t.scripts.load_model(input_dim=X_train.shape[1], file_path=file_path)

2025-08-12 21:10:44,769 - root - INFO - transformed input datasets: X_train: (429960, 63), X_val: (50000, 63), X_test: (50000, 63)


In [14]:
stockcode = '85123A'

df = df_fin.copy()
df_stockcode = df[df['stockcode'] == stockcode]
# print(df_stockcode['quantity'].unique())

# df_stockcode = data_handling.scripts.structure_missing_values(df=df_stockcode)
# df_stockcode = data_handling.scripts.handle_feature_engineering(df=df_stockcode)

# print(df_stockcode['quantity'].unique())

print(df_stockcode.head().transpose())

                                         0               49              66   \
invoiceno                             536365          536373          536375   
stockcode                             85123A          85123A          85123A   
invoicedate                     1291191960.0    1291194120.0    1291195920.0   
unitprice                               2.55            2.55            2.55   
customerid                           17850.0         17850.0         17850.0   
country                       United Kingdom  United Kingdom  United Kingdom   
year                                    2010            2010            2010   
year_month                                12              12              12   
day_of_week                              Wed             Wed             Wed   
sales                               2.727918        2.727918        2.727918   
product_avg_sales_last_month     7972.586667     7972.586667     7972.586667   
is_registered                           

In [15]:
import torch
X = df_stockcode.copy().drop(columns=target_col)
y = df_stockcode.copy()[target_col]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=1000, random_state=random_state, shuffle=True)

X_train = preprocessor.transform(X_train)
X_test = preprocessor.transform(X_test)

batch_size = 32
train_data_loader = t.scripts.create_torch_data_loader(X=X_train, y=y_train, batch_size=batch_size)
val_data_loader = t.scripts.create_torch_data_loader(X=X_val, y=y_val, batch_size=batch_size)
# retrain the best model
model, _ = t.scripts.train_model(
    model=model,
    optimizer=torch.optim.Adam(model.parameters(), lr=0.001),
    criterion=torch.nn.MSELoss(),
    num_epochs=50,
    min_delta=0.00001,
    patience=10,
    train_data_loader=train_data_loader,
    val_data_loader=val_data_loader,
    device_type='cpu'
)

2025-08-12 21:10:46,509 - root - INFO - ... starts epoch 1 ...
2025-08-12 21:10:46,702 - root - INFO - ... starts epoch 2 ...
2025-08-12 21:10:46,873 - root - INFO - ... starts epoch 3 ...
2025-08-12 21:10:47,048 - root - INFO - ... starts epoch 4 ...
2025-08-12 21:10:47,221 - root - INFO - ... starts epoch 5 ...
2025-08-12 21:10:47,388 - root - INFO - ... starts epoch 6 ...
2025-08-12 21:10:47,553 - root - INFO - ... starts epoch 7 ...
2025-08-12 21:10:47,718 - root - INFO - ... starts epoch 8 ...
2025-08-12 21:10:47,884 - root - INFO - ... starts epoch 9 ...
2025-08-12 21:10:48,054 - root - INFO - ... starts epoch 10 ...
2025-08-12 21:10:48,078 - root - INFO - epoch [10/50], loss: 0.4998
2025-08-12 21:10:48,223 - root - INFO - ... starts epoch 11 ...
2025-08-12 21:10:48,388 - root - INFO - ... starts epoch 12 ...
2025-08-12 21:10:48,552 - root - INFO - ... starts epoch 13 ...
2025-08-12 21:10:48,717 - root - INFO - early stopping at epoch 13 as validation loss did not improve for 10 

In [16]:
from src._utils import main_logger
import pandas as pd
import datetime

import src.model.torch_model as t

file_path = os.path.join(project_root, 'models', 'production', 'dfn_best.pth')
model = t.scripts.load_model(input_dim=X_train.shape[1], file_path=file_path)

min_price = 2
max_price = 100
NUM_PRICE_BINS = 1000
price_range = np.linspace(min_price, max_price, num=NUM_PRICE_BINS)

data = {}

customerid = data.get('customerid', 'unknown') if data else 'unknown'
try: customer_recency_days = df_stockcode.loc[df_stockcode['customerid'] == customerid, 'customer_recency_days_latest'].iloc[0] # type:ignore
except: customer_recency_days = 365
try: customer_total_spend_ltm = df_stockcode.loc[df_stockcode['customerid'] == customerid, 'customer_total_spend_ltm_latest'].iloc[0] # type:ignore
except: customer_total_spend_ltm = 0
try: customer_freq_ltm =  df_stockcode.loc[df_stockcode['customerid'] == customerid, 'customer_freq_ltm_latest'].iloc[0] # type:ignore
except: customer_freq_ltm = 0

new_data = {
    'invoicedate': [np.datetime64(datetime.datetime.now())] * NUM_PRICE_BINS,
    'invoiceno': [data.get('invoiceno', np.nan)] * NUM_PRICE_BINS,
    'stockcode': [stockcode] * NUM_PRICE_BINS,
    'sales': [np.nan] * NUM_PRICE_BINS,
    'customerid': [customerid] * NUM_PRICE_BINS,
    'country': [data.get('country', df_stockcode.loc[0, 'country']) if df_stockcode is not None else np.nan] * NUM_PRICE_BINS,
    'unitprice': price_range,
    'product_avg_sales_last_month': [df_stockcode.loc[0, 'product_avg_sales_last_month'] if df_stockcode is not None else 0] * NUM_PRICE_BINS,
    'is_registered': [True if customerid else False] * NUM_PRICE_BINS,
    'customer_recency_days': [customer_recency_days] * NUM_PRICE_BINS,
    'customer_total_spend_ltm': [customer_total_spend_ltm] * NUM_PRICE_BINS,
    'customer_freq_ltm': [customer_freq_ltm] * NUM_PRICE_BINS,
    'is_return': [False] * NUM_PRICE_BINS,
}
new_df = pd.DataFrame(new_data)

# add dt related features
new_df['year'] = new_df['invoicedate'].dt.year
new_df['year_month'] = new_df['invoicedate'].dt.to_period('M')
new_df['day_of_week'] = new_df['invoicedate'].dt.strftime('%a')
new_df['invoicedate'] = new_df['invoicedate'].astype(int) / 10 ** 9


# transform input data
target_col = 'sales'
X = new_df.copy().drop(target_col, axis=1)
X = X.sample(frac=1).reset_index(drop=True)
if preprocessor: X = preprocessor.transform(X)


model.eval()
input_tensor = torch.tensor(X, dtype=torch.float32)
epsilon = 1e-10
with torch.inference_mode():
    y_pred = model(input_tensor)
    y_pred = y_pred.cpu().numpy().flatten()
    y_pred_actual = np.exp(y_pred + epsilon)
    main_logger.info(f"primary model's prediction for stockcode {stockcode} - actual sales ${y_pred_actual}")


df_ = new_df.copy()
df_['sales'] = y_pred_actual
df_ = df_.sort_values(by='unitprice')

optimal_row = df_.loc[df_['sales'].idxmax()]
optimal_price = optimal_row['unitprice']
best_sales = optimal_row['sales']

all_outputs = []
for _, row in df_.iterrows():
    current_output = {
        "stockcode": stockcode,
        "unit_price": float(row['unitprice']),
        "predicted_sales": float(row['sales']),
        "optimal_unit_price": float(optimal_price), # type: ignore
        "max_predicted_sales": float(best_sales) * 30, # type: ignore
    }
    all_outputs.append(current_output)

    # print(float(row['quantity'] * row['unitprice']))

print(optimal_price)

2025-08-12 21:10:48,744 - root - INFO - primary model's prediction for stockcode 85123A - actual sales $[0.4796398  0.48732165 0.47837877 0.47809157 0.48174906 0.4791195
 0.48831302 0.4840129  0.47951803 0.48376697 0.4884032  0.48087105
 0.48652327 0.48814383 0.48572615 0.4819605  0.48684925 0.47968423
 0.48007205 0.4811931  0.48613012 0.48395708 0.47776052 0.47887617
 0.48435953 0.48724285 0.48595056 0.48416936 0.48598424 0.47885397
 0.4876031  0.47753987 0.48752427 0.47869936 0.47856668 0.4799391
 0.48384523 0.48663563 0.47982812 0.4842813  0.48569253 0.4855579
 0.47923025 0.48361072 0.48485193 0.4884597  0.4808266  0.48308602
 0.48632112 0.4845722  0.48028275 0.4787656  0.48166013 0.48194942
 0.48543456 0.48422527 0.4818492  0.47773832 0.47908634 0.4827514
 0.48272917 0.4811598  0.48173797 0.48665816 0.48602897 0.47778255
 0.48338744 0.48054913 0.4820162  0.48157114 0.48488554 0.4777716
 0.478434   0.47955126 0.4835102  0.47996125 0.47896466 0.48413587
 0.48021632 0.48659062 0.48295

80.77277277277277


In [17]:
## comment out 

# import matplotlib.pyplot as plt

# plt.figure(figsize=(10, 6)) 
# plt.scatter(df_['unitprice'], df_['sales'], color='blue', label='Predicted Sales')

# # Highlight the optimal price point on the graph
# plt.scatter(optimal_price, best_sales, color='red', s=100, zorder=5, label='Optimal Price Point')

# # Add labels and a title for clarity
# plt.xlabel('Unit Price', fontsize=12)
# plt.ylabel('Predicted Sales', fontsize=12)
# plt.title('Predicted Sales vs. Unit Price', fontsize=14)
# plt.legend()
# plt.grid(True)
# plt.xticks(df_['unitprice']) # Ensure all price points are shown on the x-axis

# # Annotate the optimal point with its values
# plt.annotate(
#     f'Optimal Price: ${optimal_price}\nMax Sales: {best_sales}',
#     xy=(optimal_price, best_sales),
#     xytext=(optimal_price + 2, best_sales + 100),
#     arrowprops=dict(facecolor='black', shrink=0.05),
#     fontsize=10
# )
# plt.savefig('my_plot.png')
# plt.show()